In [2]:
import polars as pl

# 1. Read the TSV file
# Note: quote_char=None is often safer for raw scraping data if quotes aren't strictly used for escaping
df = pl.read_csv("p2000_2026-01-03-10:40.tsv", separator='\t', quote_char=None)

# 2. Convert 'Timestamp' to actual datetime objects
# Format is DD-MM-YYYY HH:MM:SS
df = df.with_columns(
    pl.col("Timestamp").str.to_datetime("%d-%m-%Y %H:%M:%S")
)

# 3. Basic Stats
print(f"Total Rows: {df.height}")
print(f"Total Columns: {df.width}")
print("-" * 40)

# 4. Datetime Range
min_ts = df["Timestamp"].min()
max_ts = df["Timestamp"].max()
duration = max_ts - min_ts

print(f"Start Time: {min_ts}")
print(f"End Time:   {max_ts}")
print(f"Duration:   {duration}")
print("-" * 40)

# 5. Top 20 Unique Values for Text Columns
# We filter for String type columns specifically
string_cols = [name for name, dtype in df.schema.items() if dtype == pl.String]

for col_name in string_cols:
    print(f"\nTop 20 frequencies for column: '{col_name}'")
    
    # Calculate value counts, sort descending, and take top 20
    top_20 = (
        df[col_name]
        .value_counts()
        .sort("count", descending=True)
        .head(20)
    )
    
    print(top_20)


Total Rows: 29999
Total Columns: 4
----------------------------------------
Start Time: 2025-12-29 02:51:20
End Time:   2026-01-03 10:35:20
Duration:   5 days, 7:44:00
----------------------------------------

Top 20 frequencies for column: 'Service'
shape: (5, 2)
┌───────────┬───────┐
│ Service   ┆ count │
│ ---       ┆ ---   │
│ str       ┆ u32   │
╞═══════════╪═══════╡
│ Ambulance ┆ 19151 │
│ Brandweer ┆ 8942  │
│ Politie   ┆ 1891  │
│ KNRM      ┆ 14    │
│ null      ┆ 1     │
└───────────┴───────┘

Top 20 frequencies for column: 'Region'
shape: (20, 2)
┌─────────────────────────┬───────┐
│ Region                  ┆ count │
│ ---                     ┆ ---   │
│ str                     ┆ u32   │
╞═════════════════════════╪═══════╡
│ Rotterdam-Rijnmond      ┆ 4102  │
│ Haaglanden              ┆ 2767  │
│ Brabant Noord           ┆ 2574  │
│ Hollands Midden         ┆ 2343  │
│ Midden- en West Brabant ┆ 2155  │
│ …                       ┆ …     │
│ Zeeland                 ┆ 832   │
│ Gro

In [8]:
# Filter out test messages

In [7]:
import polars as pl

# 1. Read the TSV file
df = pl.read_csv("p2000_2026-01-03-10:40.tsv", separator='\t', quote_char=None)

# 2. Advanced Filtering
# - Parse dates
# - Filter out exact "TESTOPROEP MOB"
# - Filter out messages containing the whole word "test" (case-insensitive)
df_clean = (
    df
    .with_columns(pl.col("Timestamp").str.to_datetime("%d-%m-%Y %H:%M:%S"))
    .filter(
        (pl.col("Message") != "TESTOPROEP MOB") &
        (pl.col("Message") != "te5t") &
        (~pl.col("Message").str.contains(r"(?i)\btest\b"))
    )
)

# 3. Stats
print(f"Original Rows: {df.height}")
print(f"Cleaned Rows:  {df_clean.height}")
print(f"Rows Removed:  {df.height - df_clean.height}")
print("-" * 40)

# 4. Check Top 20 Remaining Messages
print("\nTop 20 Remaining Messages:")
print(
    df_clean["Message"]
    .value_counts()
    .sort("count", descending=True)
    .head(20)
)


Original Rows: 29999
Cleaned Rows:  28109
Rows Removed:  1890
----------------------------------------

Top 20 Remaining Messages:
shape: (20, 2)
┌─────────────────────────────────┬───────┐
│ Message                         ┆ count │
│ ---                             ┆ ---   │
│ str                             ┆ u32   │
╞═════════════════════════════════╪═══════╡
│ SPOED AMBU                      ┆ 38    │
│ P 1 BRT-16 (GRIP 2) (Zeer gr. … ┆ 34    │
│ P 1 BRT-10 (GRIP 1) (Zeer gr. … ┆ 34    │
│ U heeft een bericht op de tele… ┆ 33    │
│ 3301                            ┆ 30    │
│ …                               ┆ …     │
│ P 1 BRT-01 (GRIP 1) (Zeer gr. … ┆ 16    │
│ A2 DP5 Westland Galgepad NAALD… ┆ 14    │
│ 2                               ┆ 14    │
│ A2 DP5 Westland Galgepad NAALD… ┆ 13    │
│ A2 DP2 Leidschendam-Voorburg V… ┆ 13    │
└─────────────────────────────────┴───────┘


In [9]:

# 3. Aggregate per Region and Service
# First, get counts per Region + Service
stats = (
    df_clean
    .group_by(["Region", "Service"])
    .len()  # Count rows in each group
)

# 4. Pivot to create columns for each service
# We want: Region | Ambulance | Brandweer | Politie
pivot_df = (
    stats
    .pivot(
        values="len",
        index="Region",
        on="Service",
        aggregate_function="sum"
    )
    .fill_null(0) # Replace NaNs with 0
)

# 5. Clean up columns and Calculate Total
# Ensure expected columns exist (in case one service had 0 calls total)
expected_cols = ["Ambulance", "Brandweer", "Politie"]
existing_cols = pivot_df.columns

for col in expected_cols:
    if col not in existing_cols:
        pivot_df = pivot_df.with_columns(pl.lit(0).alias(col))

# Rename to English as requested and calculate Total
final_table = (
    pivot_df
    .rename({
        "Brandweer": "Firefighters",
        "Politie": "Police"
    })
    .with_columns(
        (pl.col("Ambulance") + pl.col("Firefighters") + pl.col("Police")).alias("Total")
    )
    .sort("Total", descending=True)
)

# 6. Display
print(f"Stats based on {df_clean.height} cleaned records.\n")
print(final_table)


Stats based on 28109 cleaned records.

shape: (26, 7)
┌─────────────────────────┬──────────────┬────────┬───────────┬──────┬──────┬───────┐
│ Region                  ┆ Firefighters ┆ Police ┆ Ambulance ┆ KNRM ┆ null ┆ Total │
│ ---                     ┆ ---          ┆ ---    ┆ ---       ┆ ---  ┆ ---  ┆ ---   │
│ str                     ┆ u32          ┆ u32    ┆ u32       ┆ u32  ┆ u32  ┆ u32   │
╞═════════════════════════╪══════════════╪════════╪═══════════╪══════╪══════╪═══════╡
│ Rotterdam-Rijnmond      ┆ 1005         ┆ 274    ┆ 2687      ┆ 0    ┆ 0    ┆ 3966  │
│ Haaglanden              ┆ 256          ┆ 98     ┆ 2408      ┆ 0    ┆ 0    ┆ 2762  │
│ Hollands Midden         ┆ 1121         ┆ 106    ┆ 1115      ┆ 0    ┆ 0    ┆ 2342  │
│ Midden- en West Brabant ┆ 322          ┆ 179    ┆ 1653      ┆ 0    ┆ 0    ┆ 2154  │
│ Amsterdam-Amstelland    ┆ 473          ┆ 97     ┆ 1394      ┆ 0    ┆ 0    ┆ 1964  │
│ …                       ┆ …            ┆ …      ┆ …         ┆ …    ┆ …    ┆ …     │
